# Predicting Demonstration Size on Demographic Characteristics and Salience

## GW DATS 6103: Introduction to Data Mining Final Project
### Name: Alexander D. Silberman

Note: Data is limited to 2017–2019, as COVID may substantially throw off the data.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

from census import Census
from us import states

import censusdis.data as ced
from censusdis.datasets import ACS1_PROFILE
from censusdis import states

## Data Acquisition and Wrangling

### Results Dataset: Crowd Counting Consortium Phase 1 (2017-2020)

Our results dataset: Crowd Counting Consortium Phase 1 (2017-2020): https://ash.harvard.edu/programs/crowd-counting-consortium/

In [2]:
crowd_data_orig = pd.read_csv("C:/Users/alexa/Code/GW DATS/6103 12 Intro to Data Mining/Final Project/data/crowd_counting_consortium_2017-2020/ccc_compiled_20172020.csv",encoding_errors="replace")
crowd_data_orig.head(5)

C:\Users\alexa\AppData\Local\Temp\ipykernel_46648\1640917472.py:1: DtypeWarning: Columns (6,22,31,32,33,34,35,36,37,38,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  crowd_data_orig = pd.read_csv("C:/Users/alexa/Code/GW DATS/6103 12 Intro to Data Mining/Final Project/data/crowd_counting_consortium_2017-2020/ccc_compiled_20172020.csv",encoding_errors="replace")


,date,locality,state,location_detail,online,type,macroevent,actors,claims,valence,...,source_28,source_29,source_30,notes,lat,lon,resolved_locality,resolved_county,resolved_state,fips_code
0,2017-01-01,Washington,DC,Lafayette Square Park,0.0,vigil,NaN,NaN,"for banning nuclear weapons, for peace",0.0,...,NaN,NaN,NaN,White House Peace Vigil continuous since June ...,38.907192,-77.036871,Washington,District of Columbia,DC,11001.0
1,2017-01-01,Mankato,MN,NaN,0.0,vigil,NaN,Peace Vigil Mankato,for peace,0.0,...,NaN,NaN,NaN,every Sunday since 2001,44.163578,-93.999400,Mankato,Blue Earth County,MN,27013.0
2,2017-01-01,Minneapolis,MN,U.S. Bank Stadium,0.0,protest; banner drop,NaN,general protestors,"against the Dakota Access Pipeline, for indige...",1.0,...,NaN,NaN,NaN,hung banner from stadium roof during NFL game,44.977753,-93.265011,Minneapolis,Hennepin County,MN,27053.0
3,2017-01-01,Little Compton,RI,Town Green,0.0,vigil,NaN,Sakonnet Peace Alliance,"for peace, for gun control, for climate action",1.0,...,NaN,NaN,NaN,every Sunday since 2003,41.510103,-71.171156,Little Compton,Newport County,RI,44005.0
4,2017-01-01,Oak Ridge,TN,Y-12 National Security Complex,0.0,vigil,NaN,Oak Ridge Environmental Peace Alliance,for abolishing nuclear weapons,0.0,...,NaN,NaN,NaN,every Sunday since the late 1990s,36.010356,-84.269645,Oak Ridge,Anderson County,TN,47001.0


Keep only data with a crowd size, information about what the crowd gathered for, and a FIPS code, and which aren't online.

In [3]:
crowd_data = crowd_data_orig[(crowd_data_orig["issues"].notnull()) & 
                             (crowd_data_orig["size_mean"].notnull()) & 
                             (crowd_data_orig["fips_code"].notnull()) &
                             (crowd_data_orig["online"] == 0)]
crowd_data.isna().sum()

date                    0
locality                0
state                   0
location_detail      2774
online                  0
                     ... 
lon                     2
resolved_locality     345
resolved_county      1929
resolved_state         10
fips_code               0
Length: 62, dtype: int64

In [4]:
# source_cols = np.array(["source_"]*30) + np.arange(1,31).astype(str)
# crowd_data.drop(columns=source_cols, inplace=True)
# crowd_data.head(5)
# crowd_data["fips_code"] = crowd_data["fips_code"].astype(np.int16)
# crowd_data["fips_code"]

crowd_data["year"] = crowd_data["date"].apply(lambda x:x[:4]).astype(int)
crowd_data["year"]


C:\Users\alexa\AppData\Local\Temp\ipykernel_46648\3801195337.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crowd_data["year"] = crowd_data["date"].apply(lambda x:x[:4]).astype(int)


2        2017
8        2017
13       2017
14       2017
16       2017
         ... 
72155    2020
72156    2020
72167    2020
72168    2020
72172    2020
Name: year, Length: 33811, dtype: int64

Limit to the years 2017 and 2018

In [5]:
crowd_data = crowd_data[(crowd_data["year"] == 2017) | (crowd_data["year"] == 2018)]

In [6]:
print(crowd_data["fips_code"].isna().sum(), "out of", len(crowd_data))

0 out of 16764


0 unresolved localities out of 16768; we need not discard any data.

In [7]:
# crowd_data = crowd_data[crowd_data["fips_code"].notna()]
# crowd_data["fips_code"] = crowd_data["fips_code"].astype(int)
# crowd_data["fips_code"]

Reindex to requisite columns

In [8]:
crowd_data = crowd_data.reindex(columns=["year","fips_code","type","issues","size_mean","size_cat"])
crowd_data

,year,fips_code,type,issues,size_mean,size_cat
2,2017,27053.0,protest; banner drop,banking and finance; economy; energy; environm...,2.0,1
8,2017,12086.0,demonstration,policing,18.0,1
13,2017,26161.0,protest,education; racism,200.0,2
14,2017,38085.0,protest; occupying land,energy; environment,300.0,2
16,2017,36061.0,protest,presidency,2.0,1
...,...,...,...,...,...,...
32405,2018,37081.0,protest,foreign affairs,150.0,2
32413,2018,1101.0,protest,policing; racism,12.0,1
32415,2018,17031.0,protest,housing,20.0,1
32424,2018,30087.0,protest,women's rights,100.0,2


Correct mislabeled `type`s

In [37]:
crowd_data_type = crowd_data["type"].str.lower()#.str.replace(r"\s+", " ", regex=True)

crowd_data_type_repl_dict = {"nat'l":"national",
                             ",":";", "/":";", " and ":";", "'":";", ":":";",
                             "counter-protest":"counterprotest",
                             "demonstraton":"demonstration", "demonstation":"demonstration", "demonstrations":"demonstration", "demonstarion":"demonstration", 
                             "demonstratin":"demonstration", "deomnstration":"demonstration", "demonstartion":"demonstration",
                             "flashmob":"flash mob",
                             "marach":"march",
                             "protests":"protest", "protestors":"protest", "protesst":"protest", "protest0":"protest",
                             "raly":"rally", "rallies":"rally",
                             "walk in":"walk-in","walkin":"walk-in",
                             "walk out":"walk-out", "walkout":"walk-out","walkut":"walk-out"
                             }

for key, value in crowd_data_type_repl_dict.items():
    crowd_data_type = crowd_data_type.str.replace(key, value)
    
crowd_data_type = crowd_data_type.str.split(";").explode().str.lstrip().str.rstrip()
crowd_data_type = crowd_data_type.astype(str)
crowd_data_type = crowd_data_type.loc[(crowd_data_type != "nan") & (crowd_data_type != "m") & (crowd_data_type != "0.0")]
crowd_data_type.unique()

array(['protest', 'banner drop', 'demonstration', 'occupying land',
       'strike', 'vigil', 'rally', 'direct action', 'march',
       'flag burning', 'block highways', 'sign held on crane',
       'counterprotest', 'sit-in', 'standoff', 'dance', 'blockade',
       'nonviolent blockade', 'road blockade', 'mass mooning',
       'kindness march', 'civil disobedience', 'walk-out',
       'human barricade', 'block streets', 'sit-in demonstration',
       'peace march', 'protest (armed)', 'hunger strike', 'sick out',
       'boycott', 'swim protest', 'die-in', 'boycott classes',
       'resignation', 'turn backs to speaker', 'protest resignation',
       'die in', 'art projection', 'sing-in', 'skits', 'flash mob',
       'flotilla', 'counter protest', 'block interactions',
       'light rail stop', 'block highway', 'superhero rally',
       'aircraft flyover', 'death train', 'street blockade', 'encampment',
       '"drive"', 'railroad blockade', 'pray-in', 'picket',
       'railway blockad

Onehot encode `type`

In [44]:
type_onehot = pd.get_dummies(crowd_data_type).groupby(crowd_data_type.index).any()
type_onehot

crowd_data.drop(columns="type").join(type_onehot)


# crowd_census_data_issues_exploded = crowd_census_data["issues"].explode()
# problems_onehot = pd.get_dummies(crowd_census_data_problems_exploded).groupby(crowd_census_data_problems_exploded.index).any()
# problems_onehot

,year,fips_code,issues,size_mean,size_cat,"""drive""","""kookout""",aircraft flyover,art projection,banner drop,...,strike,superhero rally,support walk-out,swim protest,teach in,turn backs to speaker,vigil,walk,walk-in,walk-out
2,2017,27053.0,banking and finance; economy; energy; environm...,2.0,1,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
8,2017,12086.0,policing,18.0,1,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
13,2017,26161.0,education; racism,200.0,2,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
14,2017,38085.0,energy; environment,300.0,2,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
16,2017,36061.0,presidency,2.0,1,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32405,2018,37081.0,foreign affairs,150.0,2,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
32413,2018,1101.0,policing; racism,12.0,1,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
32415,2018,17031.0,housing,20.0,1,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
32424,2018,30087.0,women's rights,100.0,2,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


### American Community Survey

In [9]:
api_key = "29e7dfea2f8b253a0a10ccd9626f78e49f4f0a4f"

def regex_filter(string, myregex, inverse=False):
    if string:
        mo = re.search(myregex, string)
        if inverse:
            if mo:
                return False
            else:
                return True
        else:
            if mo:
                return True
            else:
                return False
    else:
        return False

census_vars_orig = ced.variables.search(ACS1_PROFILE, 2017)
census_vars = census_vars_orig[(census_vars_orig["GROUP"] != "N/A")]
census_vars_pcts = census_vars[census_vars["VARIABLE"].apply(regex_filter,myregex=r"PE$")]
census_vars_pcts_no_ratio = census_vars_pcts[census_vars_pcts["LABEL"].apply(regex_filter,myregex=r"Sex\sratio",inverse=True)]

census_vars_agg = census_vars_pcts_no_ratio[census_vars_pcts_no_ratio["LABEL"].apply(regex_filter,myregex=r"(Median)|(Mean)|(Average)|(Per\s)|(\srate)")]
census_vars_agg[["VARIABLE","LABEL"]]

census_vars_pcts_no_ratio_agg = census_vars_pcts_no_ratio.drop(census_vars_agg.index)

census_vars_agg_ests = census_vars_agg["VARIABLE"].apply(lambda x:x.replace("PE","E"))

In [10]:
census_vars_to_drop = ["DP05_0033PE", # same as DP05_0001PE (Total Population)
                       "DP05_0086PE" # total housing units—replace with DP05_0086E

                       ]

census_vars_to_add = ["DP05_0086E"
                      ]

census_vars_pcts_no_ratio_agg_drop = census_vars_pcts_no_ratio_agg["VARIABLE"][~census_vars_pcts_no_ratio_agg["VARIABLE"].isin(census_vars_to_drop)]

census_vars_final = pd.concat([pd.Series(["NAME"]), 
                               pd.concat([census_vars_pcts_no_ratio_agg_drop, 
                                          census_vars_agg_ests, 
                                          pd.Series(census_vars_to_add)], 
                                         ignore_index=True).sort_values()], 
                              ignore_index=True)

census_vars_final

0               NAME
1      DP02PR_0001PE
2      DP02PR_0002PE
3      DP02PR_0003PE
4      DP02PR_0004PE
           ...      
665      DP05_0085PE
666       DP05_0086E
667      DP05_0087PE
668      DP05_0088PE
669      DP05_0089PE
Length: 670, dtype: object

Solely request percents. The list of variables includes what should be the total population as a percent—`DP05_0001PE`—but, as that would always be 100%, the wise people at the US Census Bureau made such equal to `DP05_0001E`, the total population estimate, necessitating no further adjustment.

In [11]:
ced.variables.group_tree(ACS1_PROFILE, 2017,"DP02")

+ Estimate
    + HOUSEHOLDS BY TYPE
        + Total households (DP02_0001E)
            + Family households (families) (DP02_0002E)
                + With own children of the householder under 18 years (DP02_0003E)
                + Married-couple family (DP02_0004E)
                    + With own children of the householder under 18 years (DP02_0005E)
                + Male householder, no wife present, family (DP02_0006E)
                    + With own children of the householder under 18 years (DP02_0007E)
                + Female householder, no husband present, family (DP02_0008E)
                    + With own children of the householder under 18 years (DP02_0009E)
            + Nonfamily households (DP02_0010E)
                + Householder living alone (DP02_0011E)
                    + 65 years and over (DP02_0012E)
        + Households with one or more people under 18 years (DP02_0013E)
        + Households with one or more people 65 years and over (DP02_0014E)
        + Aver

In [12]:
census_data = [ced.download(ACS1_PROFILE, year, census_vars_final, 
                           metropolitan_statistical_area_micropolitan_statistical_area ="*", 
                           api_key=api_key
                           ).set_index("METROPOLITAN_STATISTICAL_AREA_MICROPOLITAN_STATISTICAL_AREA")
               for year in range(2017,2019)]


census_data = pd.concat(census_data, keys = np.arange(2017,2019), names=["year"])
census_data

NAME  \
year METROPOLITAN_STATISTICAL_AREA_MICROPOLITAN_STAT...                                         
2017 44220                                                         Springfield, OH Metro Area   
     44300                                                       State College, PA Metro Area   
     44340                                                          Statesboro, GA Micro Area   
     44420                                                 Staunton-Waynesboro, VA Metro Area   
     44620                                                       Stevens Point, WI Micro Area   
...                                                                                       ...   
2018 31820                                                           Manitowoc, WI Micro Area   
     31860                                               Mankato-North Mankato, MN Metro Area   
     31900                                                           Mansfield, OH Metro Area   
     31940                                                        Marinette, WI-MI Micro Area   
     31980                                                              Marion, IN Micro Area   

                                                         DP02PR_0001PE  \
year METROPOLITAN_STATISTICAL_AREA_MICROPOLITAN_STAT...                  
2017 44220                                                         NaN   
     44300                                                         NaN   
     44340                                                         NaN   
     44420                                                         NaN   
     44620                                                         NaN   
...                                                                ...   
2018 31820                                                         NaN   
     31860                                                         NaN   
     31900                                                         NaN   
     31940                                                         NaN   
     31980                                                         NaN   

                                                         DP02PR_0002PE  \
year METROPOLITAN_STATISTICAL_AREA_MICROPOLITAN_STAT...                  
2017 44220                                                         NaN   
     44300                                                         NaN   
     44340                                                         NaN   
     44420                                                         NaN   
     44620                                                         NaN   
...                                                                ...   
2018 31820                                                         NaN   
     31860                                                         NaN   
     31900                                                         NaN   
     31940                                                         NaN   
     31980                                                         NaN   

                                                         DP02PR_0003PE  \
year METROPOLITAN_STATISTICAL_AREA_MICROPOLITAN_STAT...                  
2017 44220                                                         NaN   
     44300                                                         NaN   
     44340                                                         NaN   
     44420                                                         NaN   
     44620                                                         NaN   
...                                                                ...   
2018 31820                                                         NaN   
     31860                                                         NaN   
     31900                                                         NaN   
     31940                                                         NaN   
     31980   

In [13]:
census_data.columns[census_data.isna().sum() == 1037]

Index(['DP02PR_0114PE', 'DP02PR_0115PE', 'DP02PR_0116PE', 'DP02PR_0117PE',
       'DP02PR_0118PE', 'DP02PR_0119PE', 'DP02PR_0120PE', 'DP02PR_0121PE',
       'DP02PR_0122PE', 'DP02PR_0123PE', 'DP02PR_0124PE', 'DP02PR_0125PE',
       'DP02PR_0126PE', 'DP02PR_0127PE', 'DP02PR_0128PE', 'DP02PR_0129PE',
       'DP02PR_0130PE', 'DP02PR_0131PE', 'DP02PR_0132PE', 'DP02PR_0133PE',
       'DP02PR_0134PE', 'DP02PR_0135PE', 'DP02PR_0136PE', 'DP02PR_0137PE',
       'DP02PR_0138PE', 'DP02PR_0139PE', 'DP02PR_0140PE', 'DP02PR_0141PE',
       'DP02PR_0142PE', 'DP02PR_0143PE', 'DP02PR_0144PE', 'DP02PR_0145PE',
       'DP02PR_0146PE', 'DP02PR_0147PE', 'DP02PR_0148PE', 'DP02PR_0149PE',
       'DP04_0116PE', 'DP04_0125PE', 'DP04_0135PE', 'DP04_0143PE'],
      dtype='object')

In [14]:
# metro_micro_area_text = re.compile(r"(\s(Metro|Micro)\sArea)$")
# census_data["NAME"].apply(lambda x: metro_micro_area_text.sub("",x))
# # re.sub(string="New York Metro Area", pattern=r"(Metro|Micro)\sArea$", repl="hello")

### Most Important Problem Dataset, Second Release (2024)

The aggregated data may be found here: https://williamslaro.github.io/talks/dataset2. Previously, the individual data was used, necessitating much preprocessing.

In [39]:
issue_data = pd.read_csv("C:\\Users\\alexa\\Code\\GW DATS\\6103 12 Intro to Data Mining\\Final Project\\data\\MIPD-2024-Aggregate\\MIPDV2-All-year.csv",encoding="ANSI",
                         index_col=["year","problem"])
issue_data.head(20)

cat  numsurveys  numrespondents       perc         catname  \
year problem                                                               
1939 1          1           3            8989  45.401001  MIPD (General)   
     2          1           3            8989   6.861000  MIPD (General)   
     3          1           3            8989   0.200000  MIPD (General)   
     4          1           3            8989   0.554000  MIPD (General)   
     5          1           3            8989   4.629000  MIPD (General)   
     6          1           3            8989  33.779999  MIPD (General)   
     7          1           3            8989   0.056000  MIPD (General)   
     8          1           3            8989   0.044000  MIPD (General)   
     9          1           3            8989   1.432000  MIPD (General)   
     10         1           3            8989   4.123000  MIPD (General)   
     11         1           3            8989   0.700000  MIPD (General)   
     12         1           3            8989   0.168000  MIPD (General)   
     13         1           3            8989   1.461000  MIPD (General)   
     14         1           3            8989   0.078000  MIPD (General)   
     15         1           3            8989   0.512000  MIPD (General)   
1940 1          1           1            2907  24.200001  MIPD (General)   
     2          1           1            2907   3.474000  MIPD (General)   
     3          1           1            2907   0.069000  MIPD (General)   
     4          1           1            2907   0.172000  MIPD (General)   
     5          1           1            2907  13.152000  MIPD (General)   

                                             catdesc  \
year problem                                           
1939 1        general problem categories in the MIPD   
     2        general problem categories in the MIPD   
     3        general problem categories in the MIPD   
     4        general problem categories in the MIPD   
     5        general problem categories in the MIPD   
     6        general problem categories in the MIPD   
     7        general problem categories in the MIPD   
     8        general problem categories in the MIPD   
     9        general problem categories in the MIPD   
     10       general problem categories in the MIPD   
     11       general problem categories in the MIPD   
     12       general problem categories in the MIPD   
     13       general problem categories in the MIPD   
     14       general problem categories in the MIPD   
     15       general problem categories in the MIPD   
1940 1        general problem categories in the MIPD   
     2        general problem categories in the MIPD   
     3        general problem categories in the MIPD   
     4        general problem categories in the MIPD   
     5        general problem categories in the MIPD   

                                   problemname  \
year problem                                     
1939 1                                 Economy   
     2                           Social Policy   
     3                                  Rights   
     4                           Public Safety   
     5                           Fiscal Policy   
     6                          Foreign Policy   
     7        International Economic Relations   
     8                             Environment   
     9                                Morality   
     10                               Politics   
     11                           Youth Issues   
     12                                 Groups   
     13                          Other and All   
     14                                    NaN   
     15                     Don't Know/Refused   
1940 1                                 Economy   
     2                           Social Policy   
     3                                  Rights   
     4                           Public Safety   
     5                           Fiscal Policy   

       

Limit to relevant years

In [16]:
issue_data = issue_data.loc[[2017,2018]]
issue_data

cat  numsurveys  numrespondents       perc         catname  \
year problem                                                               
2017 1          1           2            1999  19.669001  MIPD (General)   
     2          1           2            1999  16.531000  MIPD (General)   
     3          1           2            1999   4.969000  MIPD (General)   
     4          1           2            1999   1.275000  MIPD (General)   
     5          1           2            1999  15.047000  MIPD (General)   
     6          1           2            1999  11.080000  MIPD (General)   
     7          1           2            1999   0.222000  MIPD (General)   
     8          1           2            1999   2.157000  MIPD (General)   
     9          1           2            1999   5.814000  MIPD (General)   
     10         1           2            1999  14.678000  MIPD (General)   
     11         1           2            1999   0.426000  MIPD (General)   
     12         1           2            1999   0.000000  MIPD (General)   
     13         1           2            1999   3.186000  MIPD (General)   
     14         1           2            1999   0.269000  MIPD (General)   
     15         1           2            1999   4.676000  MIPD (General)   
2018 1          1           2            1983  14.319000  MIPD (General)   
     2          1           2            1983  11.224000  MIPD (General)   
     3          1           2            1983   8.407000  MIPD (General)   
     4          1           2            1983   3.457000  MIPD (General)   
     5          1           2            1983  15.009000  MIPD (General)   
     6          1           2            1983  12.685000  MIPD (General)   
     7          1           2            1983   0.282000  MIPD (General)   
     8          1           2            1983   1.959000  MIPD (General)   
     9          1           2            1983   5.000000  MIPD (General)   
     10         1           2            1983  17.754000  MIPD (General)   
     11         1           2            1983   0.707000  MIPD (General)   
     12         1           2            1983   0.000000  MIPD (General)   
     13         1           2            1983   4.345000  MIPD (General)   
     14         1           2            1983   0.125000  MIPD (General)   
     15         1           2            1983   4.728000  MIPD (General)   

                                             catdesc  \
year problem                                           
2017 1        general problem categories in the MIPD   
     2        general problem categories in the MIPD   
     3        general problem categories in the MIPD   
     4        general problem categories in the MIPD   
     5        general problem categories in the MIPD   
     6        general problem categories in the MIPD   
     7        general problem categories in the MIPD   
     8        general problem categories in the MIPD   
     9        general problem categories in the MIPD   
     10       general problem categories in the MIPD   
     11       general problem categories in the MIPD   
     12       general problem categories in the MIPD   
     13       general problem categories in the MIPD   
     14       general problem categories in the MIPD   
     15       general problem categories in the MIPD   
2018 1        general problem categories in the MIPD   
     2        general problem categories in the MIPD   
     3        general problem categories in the MIPD   
     4        general problem categories in the MIPD   
     5        general problem categories in the MIPD   
     6        general problem categories in the MIPD   
     7        general problem categories in the MIPD   
     8        general problem categories in the MIPD   
     9        general problem categories in the MIPD   
     10       general problem categories in the MIPD   
     11       general problem categories in the MIPD   
     12      

### Merge the Datasets

In [17]:
msa_fips_crosswalk = pd.read_excel("C:\\Users\\alexa\\Code\\GW DATS\\6103 12 Intro to Data Mining\\Final Project\\data\\list1_2023.xlsx",skiprows=2,dtype=str)
msa_fips_crosswalk.head()

,CBSA Code,Metropolitan Division Code,CSA Code,CBSA Title,Metropolitan/Micropolitan Statistical Area,Metropolitan Division Title,CSA Title,County/County Equivalent,State Name,FIPS State Code,FIPS County Code,Central/Outlying County
0,10100,NaN,NaN,"Aberdeen, SD",Micropolitan Statistical Area,NaN,NaN,Brown County,South Dakota,46,013,Central
1,10100,NaN,NaN,"Aberdeen, SD",Micropolitan Statistical Area,NaN,NaN,Edmunds County,South Dakota,46,045,Outlying
2,10140,NaN,NaN,"Aberdeen, WA",Micropolitan Statistical Area,NaN,NaN,Grays Harbor County,Washington,53,027,Central
3,10180,NaN,101,"Abilene, TX",Metropolitan Statistical Area,NaN,"Abilene-Sweetwater, TX",Callahan County,Texas,48,059,Outlying
4,10180,NaN,101,"Abilene, TX",Metropolitan Statistical Area,NaN,"Abilene-Sweetwater, TX",Jones County,Texas,48,253,Outlying


In [18]:
msa_fips_crosswalk["FIPS Code"] = msa_fips_crosswalk["FIPS State Code"] + msa_fips_crosswalk["FIPS County Code"]

Drop NaN FIPS codes in preparation for merging.

In [19]:
msa_fips_crosswalk = msa_fips_crosswalk[msa_fips_crosswalk["FIPS Code"].notna()]
msa_fips_crosswalk["FIPS Code"] = msa_fips_crosswalk["FIPS Code"].astype(int)
msa_fips_crosswalk["FIPS Code"]

0       46013
1       46045
2       53027
3       48059
4       48253
        ...  
1910     6101
1911     6115
1912     4027
1913    39119
1914    48505
Name: FIPS Code, Length: 1915, dtype: int64

In [20]:
crowd_data_msa = crowd_data.merge(msa_fips_crosswalk[["FIPS Code", "CBSA Code"]], how="left", left_on="fips_code", right_on="FIPS Code")
crowd_data_msa.head()


,year,fips_code,type,issues,size_mean,size_cat,FIPS Code,CBSA Code
0,2017,27053.0,protest; banner drop,banking and finance; economy; energy; environm...,2.0,1,27053.0,33460
1,2017,12086.0,demonstration,policing,18.0,1,12086.0,33100
2,2017,26161.0,protest,education; racism,200.0,2,26161.0,11460
3,2017,38085.0,protest; occupying land,energy; environment,300.0,2,NaN,NaN
4,2017,36061.0,protest,presidency,2.0,1,36061.0,35620


In [21]:
crowd_census_data = crowd_data_msa.merge(census_data, how="inner", 
                                         left_on = ["year","CBSA Code"], right_on = ["year","METROPOLITAN_STATISTICAL_AREA_MICROPOLITAN_STATISTICAL_AREA"])

In [22]:
crowd_census_data.head()

,year,fips_code,type,issues,size_mean,size_cat,FIPS Code,CBSA Code,NAME,DP02PR_0001PE,...,DP05_0080PE,DP05_0081PE,DP05_0082PE,DP05_0083PE,DP05_0084PE,DP05_0085PE,DP05_0086E,DP05_0087PE,DP05_0088PE,DP05_0089PE
0,2017,27053.0,protest; banner drop,banking and finance; economy; energy; environm...,2.0,1,27053.0,33460,"Minneapolis-St. Paul-Bloomington, MN-WI Metro ...",NaN,...,6.7,0.0,0.1,2.7,0.1,2.6,1447758,2591001,49.0,51.0
1,2017,12086.0,demonstration,policing,18.0,1,12086.0,33100,"Miami-Fort Lauderdale-West Palm Beach, FL Metr...",NaN,...,2.4,0.0,0.4,1.3,0.2,1.1,2534993,3924471,47.4,52.6
2,2017,26161.0,protest,education; racism,200.0,2,26161.0,11460,"Ann Arbor, MI Metro Area",NaN,...,9.0,0.0,0.3,4.1,0.1,4.0,150397,278978,49.2,50.8
3,2017,36061.0,protest,presidency,2.0,1,36061.0,35620,"New York-Newark-Jersey City, NY-NJ-PA Metro Area",NaN,...,11.3,0.0,0.7,1.7,0.2,1.5,7983770,13563604,47.2,52.8
4,2017,11001.0,protest,presidency,3.0,1,11001.0,47900,"Washington-Arlington-Alexandria, DC-VA-MD-WV M...",NaN,...,10.2,0.0,0.4,3.3,0.2,3.1,2354827,4155063,48.2,51.8


#### Merge on Issue Salience

Some crowds are gathered for issues that fit into multiple categories, organized in alphabetical order. If this is the case, how should they be merged?

In [23]:
crowd_census_data["issues"] = crowd_census_data["issues"].apply(lambda x:np.array(x.split("; ")))

In [24]:
crowd_census_data["issues"]

0        [banking and finance, economy, energy, environ...
1                                               [policing]
2                                      [education, racism]
3                                             [presidency]
4                                             [presidency]
                               ...                        
14880                                         [presidency]
14881                                  [education, racism]
14882                                    [foreign affairs]
14883                                   [policing, racism]
14884                                            [housing]
Name: issues, Length: 14885, dtype: object

Perhaps solely the issue with the highest salience should be accounted for? Or, perhaps, the pcts should be added, representing the salience of anyone interested in at least one of the issues?

"The percentages can therefore be interpreted as the percentage of the electorate who answered the question and selected that category." -MIPDV2-Codebook p. 5

The latter will be used.

In [25]:
np.unique(np.concat(crowd_census_data["issues"]))

array(['animal rights', 'banking and finance', 'civil rights',
       'corruption', 'covid', 'criminal justice', 'democracy',
       'development', 'disability rights', 'drugs', 'economy',
       'education', 'energy', 'environment', 'foreign affairs',
       'free speech', 'guns', 'healthcare', 'housing', 'immigration',
       'indigenous peoples', 'judiciary', 'labor', 'legislative',
       'lgbtqia', 'military', 'patriotism', 'policing', 'presidency',
       'racism', 'religion', 'reproductive rights', 'science',
       'sexual violence', 'sports', 'taxes', 'transportation',
       "women's rights"], dtype='<U19')

All the issues in the CCC data. Each is to be mapped to a problem code in the MIPD data. This will be done by hand.

But why is `covid` there? This should be limited to the years 2017 and 18.

In [26]:
crowd_census_data_covid = crowd_census_data["issues"].apply(lambda x:"covid" in x)
crowd_census_data[crowd_census_data_covid]

,year,fips_code,type,issues,size_mean,size_cat,FIPS Code,CBSA Code,NAME,DP02PR_0001PE,...,DP05_0080PE,DP05_0081PE,DP05_0082PE,DP05_0083PE,DP05_0084PE,DP05_0085PE,DP05_0086E,DP05_0087PE,DP05_0088PE,DP05_0089PE
4099,2017,8031.0,rally,"[covid, judiciary]",20.0,1,8031.0,19740,"Denver-Aurora-Lakewood, CO Metro Area",NaN,...,4.3,0.1,0.2,2.2,0.1,2.2,1173413,2043836,49.2,50.8
11871,2018,13121.0,protest; rally,"[covid, healthcare]",100.0,2,13121.0,12060,"Atlanta-Sandy Springs-Roswell, GA Metro Area",NaN,...,6.1,0.1,0.4,2.2,0.1,2.1,2331002,4111190,47.1,52.9
13342,2018,6075.0,protest,"[covid, energy]",11.0,1,6075.0,41860,"San Francisco-Oakland-Hayward, CA Metro Area",NaN,...,26.3,0.6,0.4,4.5,0.2,4.3,1825205,3230062,49.0,51.0


According to the data dictionary, "These are generated after data compilation by running a series of regular expressions over the claims description text" (p. 3). These claims all include the words "reopen" or "reopening"; these are false positives. As such, `covid` will manually be removed from these four so as to not impact findings.

In [27]:
crowd_census_data.loc[crowd_census_data_covid,"issues"] = crowd_census_data.loc[crowd_census_data_covid, "issues"].apply(lambda x:np.setdiff1d(x, ("covid")))
crowd_census_data.loc[crowd_census_data_covid,"issues"]

4099      [judiciary]
11871    [healthcare]
13342        [energy]
Name: issues, dtype: object

Check again for unique issues:

In [29]:
np.unique(np.concat(crowd_census_data["issues"]))

array(['animal rights', 'banking and finance', 'civil rights',
       'corruption', 'criminal justice', 'democracy', 'development',
       'disability rights', 'drugs', 'economy', 'education', 'energy',
       'environment', 'foreign affairs', 'free speech', 'guns',
       'healthcare', 'housing', 'immigration', 'indigenous peoples',
       'judiciary', 'labor', 'legislative', 'lgbtqia', 'military',
       'patriotism', 'policing', 'presidency', 'racism', 'religion',
       'reproductive rights', 'science', 'sexual violence', 'sports',
       'taxes', 'transportation', "women's rights"], dtype='<U19')

In [30]:
issue_data[["problemname","problemdesc","problemissues","problemexamples","perc"]]

problemname  \
year problem                                     
2017 1                                 Economy   
     2                           Social Policy   
     3                                  Rights   
     4                           Public Safety   
     5                           Fiscal Policy   
     6                          Foreign Policy   
     7        International Economic Relations   
     8                             Environment   
     9                                Morality   
     10                               Politics   
     11                           Youth Issues   
     12                                 Groups   
     13                          Other and All   
     14                                    NaN   
     15                     Don't Know/Refused   
2018 1                                 Economy   
     2                           Social Policy   
     3                                  Rights   
     4                           Public Safety   
     5                           Fiscal Policy   
     6                          Foreign Policy   
     7        International Economic Relations   
     8                             Environment   
     9                                Morality   
     10                               Politics   
     11                           Youth Issues   
     12                                 Groups   
     13                          Other and All   
     14                                    NaN   
     15                     Don't Know/Refused   

                                                    problemdesc  \
year problem                                                      
2017 1                 includes all mentions of economic issues   
     2                   includes all mentions of social policy   
     3        includes all mentions of civil and political r...   
     4         includes all mentions of crime and public safety   
     5            includes all mentions of budgets and spending   
     6        includes all mentions of foreign policy and ot...   
     7        includes all mentions of international economi...   
     8        includes all mentions of the environment, natu...   
     9        includes all mentions of morals, values, and r...   
     10       includes all mentions of political leaders, va...   
     11                   includes all mentions of youth issues   
     12                includes all mentions of minority groups   
     13       includes mentions of other problems or all pro...   
     14                        includes mentions of no problems   
     15       includes any mention that is related to don't ...   
2018 1                 includes all mentions of economic issues   
     2                   includes all mentions of social policy   
     3        includes all mentions of civil and political r...   
     4         includes all mentions of crime and public safety   
     5            includes all mentions of budgets and spending   
     6        includes all mentions of foreign policy and ot...   
     7        includes all mentions of international economi...   
     8        includes all mentions of the environment, natu...   
     9        includes all mentions of morals, values, and r...   
     10       includes all mentions of political leaders, va...   
     11                   includes all mentions of youth issues   
     12                includes all mentions of minority groups   
     13       includes mentions of other problems or all pro...   
     14                        includes mentions of no problems   
     15       includes any mention that is related to don't ...   

                                                  problemissues  \
year problem                                                      
2017 1        Economy (general), unemployment, inflation, gr...   
     2        Social policy (general), welfare, education, h...   
     3        Gender 

In [31]:
issue_code_dict = {'animal rights':14, 
                   'banking and finance':1, 
                   'civil rights':3,
                   'corruption':10, 
                   'criminal justice':3, 
                   'democracy':10,
                   'development':1, 
                   'disability rights':3, 
                   'drugs':4, 
                   'economy':1,
                   'education':2, 
                   'energy':8, 
                   'environment':8, 
                   'foreign affairs':6,
                   'free speech':3, 
                   'guns':4, 
                   'healthcare':2, 
                   'housing':2, 
                   'immigration':5,
                   'indigenous peoples':12, 
                   'judiciary':10, 
                   'labor':1, 
                   'legislative':10,
                   'lgbtqia':3, 
                   'military':6, 
                   'patriotism':9, 
                   'policing':3,
                   'presidency':10,
                   'racism':3, 
                   'religion':9, 
                   'reproductive rights':3, 
                   'science':10,
                   'sexual violence':4, 
                   'sports':13, 
                   'taxes':1, 
                   'transportation':2,
                   "women's rights":3
}

In [32]:
crowd_census_data["problems"] = crowd_census_data["issues"].apply(np.vectorize(lambda x: issue_code_dict[x])).apply(np.unique)
crowd_census_data_exploded = crowd_census_data.explode(column="problems")
crowd_census_data_exploded.head()

# crowd_census_data_issues_exploded = crowd_census_data["issues"].explode()
# problems_onehot = pd.get_dummies(crowd_census_data_problems_exploded).groupby(crowd_census_data_problems_exploded.index).any()
# problems_onehot

,year,fips_code,type,issues,size_mean,size_cat,FIPS Code,CBSA Code,NAME,DP02PR_0001PE,...,DP05_0081PE,DP05_0082PE,DP05_0083PE,DP05_0084PE,DP05_0085PE,DP05_0086E,DP05_0087PE,DP05_0088PE,DP05_0089PE,problems
0,2017,27053.0,protest; banner drop,"[banking and finance, economy, energy, environ...",2.0,1,27053.0,33460,"Minneapolis-St. Paul-Bloomington, MN-WI Metro ...",NaN,...,0.0,0.1,2.7,0.1,2.6,1447758,2591001,49.0,51.0,1
0,2017,27053.0,protest; banner drop,"[banking and finance, economy, energy, environ...",2.0,1,27053.0,33460,"Minneapolis-St. Paul-Bloomington, MN-WI Metro ...",NaN,...,0.0,0.1,2.7,0.1,2.6,1447758,2591001,49.0,51.0,8
0,2017,27053.0,protest; banner drop,"[banking and finance, economy, energy, environ...",2.0,1,27053.0,33460,"Minneapolis-St. Paul-Bloomington, MN-WI Metro ...",NaN,...,0.0,0.1,2.7,0.1,2.6,1447758,2591001,49.0,51.0,12
1,2017,12086.0,demonstration,[policing],18.0,1,12086.0,33100,"Miami-Fort Lauderdale-West Palm Beach, FL Metr...",NaN,...,0.0,0.4,1.3,0.2,1.1,2534993,3924471,47.4,52.6,3
2,2017,26161.0,protest,"[education, racism]",200.0,2,26161.0,11460,"Ann Arbor, MI Metro Area",NaN,...,0.0,0.3,4.1,0.1,4.0,150397,278978,49.2,50.8,2


Now, this may be merged with the issue data.

We want to keep the index, however, so that we may group by it later.

crowd_census_data_exploded

In [33]:
crowd_census_data_exploded_to_merge = crowd_census_data_exploded.reset_index()

demo_data_exploded = crowd_census_data_exploded_to_merge.merge(issue_data[["perc","problemname"]], how="inner", left_on=["year","problems"], right_on=["year","problem"])
demo_data_exploded

,index,year,fips_code,type,issues,size_mean,size_cat,FIPS Code,CBSA Code,NAME,...,DP05_0083PE,DP05_0084PE,DP05_0085PE,DP05_0086E,DP05_0087PE,DP05_0088PE,DP05_0089PE,problems,perc,problemname
0,0,2017,27053.0,protest; banner drop,"[banking and finance, economy, energy, environ...",2.0,1,27053.0,33460,"Minneapolis-St. Paul-Bloomington, MN-WI Metro ...",...,2.7,0.1,2.6,1447758,2591001,49.0,51.0,1,19.669001,Economy
1,0,2017,27053.0,protest; banner drop,"[banking and finance, economy, energy, environ...",2.0,1,27053.0,33460,"Minneapolis-St. Paul-Bloomington, MN-WI Metro ...",...,2.7,0.1,2.6,1447758,2591001,49.0,51.0,8,2.157000,Environment
2,0,2017,27053.0,protest; banner drop,"[banking and finance, economy, energy, environ...",2.0,1,27053.0,33460,"Minneapolis-St. Paul-Bloomington, MN-WI Metro ...",...,2.7,0.1,2.6,1447758,2591001,49.0,51.0,12,0.000000,Groups
3,1,2017,12086.0,demonstration,[policing],18.0,1,12086.0,33100,"Miami-Fort Lauderdale-West Palm Beach, FL Metr...",...,1.3,0.2,1.1,2534993,3924471,47.4,52.6,3,4.969000,Rights
4,2,2017,26161.0,protest,"[education, racism]",200.0,2,26161.0,11460,"Ann Arbor, MI Metro Area",...,4.1,0.1,4.0,150397,278978,49.2,50.8,2,16.531000,Social Policy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22236,14881,2018,37135.0,protest,"[education, racism]",2.0,1,37135.0,20500,"Durham-Chapel Hill, NC Metro Area",...,2.5,0.1,2.4,249694,416987,46.7,53.3,2,11.224000,Social Policy
22237,14881,2018,37135.0,protest,"[education, racism]",2.0,1,37135.0,20500,"Durham-Chapel Hill, NC Metro Area",...,2.5,0.1,2.4,249694,416987,46.7,53.3,3,8.407000,Rights
22238,14882,2018,37081.0,protest,[foreign affairs],150.0,2,37081.0,24660,"Greensboro-High Point, NC Metro Area",...,2.5,0.1,2.4,337386,558753,46.4,53.6,6,12.685000,Foreign Policy
22239,14883,2018,1101.0,protest,"[policing, racism]",12.0,1,1101.0,33860,"Montgomery, AL Metro Area",...,1.6,0.1,1.5,168054,279570,46.6,53.4,3,8.407000,Rights


In [34]:
demo_data = demo_data_exploded.drop(columns=["issues","problems","problemname"])
columns_not_perc = demo_data.columns
columns_not_perc = list(columns_not_perc.drop("perc"))
columns_not_perc
demo_data = demo_data.groupby(by=columns_not_perc, dropna=False).sum().reset_index()
demo_data

C:\Users\alexa\AppData\Local\Temp\ipykernel_45748\3685731779.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  demo_data = demo_data.groupby(by=columns_not_perc, dropna=False).sum().reset_index()
C:\Users\alexa\AppData\Local\Temp\ipykernel_45748\3685731779.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  demo_data = demo_data.groupby(by=columns_not_perc, dropna=False).sum().reset_index()
C:\Users\alexa\AppData\Local\Temp\ipykernel_45748\3685731779.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually th

,index,year,fips_code,type,size_mean,size_cat,FIPS Code,CBSA Code,NAME,DP02PR_0001PE,...,DP05_0081PE,DP05_0082PE,DP05_0083PE,DP05_0084PE,DP05_0085PE,DP05_0086E,DP05_0087PE,DP05_0088PE,DP05_0089PE,perc
0,0,2017,27053.0,protest; banner drop,2.0,1,27053.0,33460,"Minneapolis-St. Paul-Bloomington, MN-WI Metro ...",NaN,...,0.0,0.1,2.7,0.1,2.6,1447758,2591001,49.0,51.0,21.826001
1,1,2017,12086.0,demonstration,18.0,1,12086.0,33100,"Miami-Fort Lauderdale-West Palm Beach, FL Metr...",NaN,...,0.0,0.4,1.3,0.2,1.1,2534993,3924471,47.4,52.6,4.969000
2,2,2017,26161.0,protest,200.0,2,26161.0,11460,"Ann Arbor, MI Metro Area",NaN,...,0.0,0.3,4.1,0.1,4.0,150397,278978,49.2,50.8,21.500000
3,3,2017,36061.0,protest,2.0,1,36061.0,35620,"New York-Newark-Jersey City, NY-NJ-PA Metro Area",NaN,...,0.0,0.7,1.7,0.2,1.5,7983770,13563604,47.2,52.8,14.678000
4,4,2017,11001.0,protest,3.0,1,11001.0,47900,"Washington-Arlington-Alexandria, DC-VA-MD-WV M...",NaN,...,0.0,0.4,3.3,0.2,3.1,2354827,4155063,48.2,51.8,14.678000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14880,14880,2018,12099.0,protest,1.0,1,12099.0,33100,"Miami-Fort Lauderdale-West Palm Beach, FL Metr...",NaN,...,0.0,0.4,1.4,0.1,1.3,2548910,3988782,47.4,52.6,17.754000
14881,14881,2018,37135.0,protest,2.0,1,37135.0,20500,"Durham-Chapel Hill, NC Metro Area",NaN,...,0.0,0.4,2.5,0.1,2.4,249694,416987,46.7,53.3,19.631000
14882,14882,2018,37081.0,protest,150.0,2,37081.0,24660,"Greensboro-High Point, NC Metro Area",NaN,...,0.0,0.5,2.5,0.1,2.4,337386,558753,46.4,53.6,12.685000
14883,14883,2018,1101.0,protest,12.0,1,1101.0,33860,"Montgomery, AL Metro Area",NaN,...,0.0,0.1,1.6,0.1,1.5,168054,279570,46.6,53.4,8.407000
